In [1]:
import numpy as np
# import tensorflow
from keras.datasets.mnist import load_data
from abc import ABC, abstractmethod

## Abstract Model

In [2]:
class Model(ABC):
    
    @abstractmethod
    def fit(self):
        '''
        Fit model on train set
        '''
        pass

    @abstractmethod
    def evaluate(self):
        '''
        Evaluate model on validation or test set
        '''
        pass

## Feedforward NN 

In [31]:
class Network(Model):
    def __init__(self, model_dim: list) -> None: 
        '''
        Args: input: input np ndarray of shape (1,N) 
            model_dim: NN dim [748,30,60,30,10]
        '''
        self.model_dim = model_dim
        # w and b matrices excludes input layer:
        self.weights = [np.random.randn(i,j) for i,j in zip(model_dim[:-1], model_dim[1:])] # normally distributed
        self.biases = [np.random.randn(1,i) for i in model_dim[1:]] 
        self.lr_0 = 0.00001 # initial LR
        self.batch_size = 10       
        self.max_epochs = 1 
        self.evaluate_every_n_epochs = 1
        

    def forwardpass(self, x, layer, activation_func, output_unit, mode = 'train') -> tuple[np.ndarray,np.ndarray] | np.ndarray:
        '''
        calculates the activations of one layer
        Args: x: Activations from prev layer or input train/eval/test data 
        layer: layer number for which we are performing the pass
            w: weights of current layer, indexing of 1st layer(input) starts from 0
            b: baises of current layer, indexing of 1st layer(input) starts from 0
        y = x'Tw + b
        '''
        w = self.weights[layer-1] # idx correction. 0 idx is weights b/w 1st and 2nd layer.
        b = self.biases[layer-1]
    
        if layer == len(self.model_dim)-1: # if last layer, use output units instead
            activation_func = output_unit
        
        z = np.dot(x,w) + b # and not W,x?
        
        if mode == 'train':
            return (z,activation_func(z)) 

        elif mode == 'eval':
            return activation_func(z) 
        
        else: return None

    def costfunc(self, y, cost_func) -> None:
        '''
        (NOT USED)
        Calculates values of objective function during evaluation only
        MSE, RMSE, Cross Entropy etc.
        Args: y : validation data labels
            cost_func: objective function like MSE, RMSE, cross entropy
        '''

        # self.costs = cost_func(self.activations[-1],y)
        return None


    def backwardpass(self, x: np.ndarray, y: np.ndarray,
                     optimzation_func, cost_func_deriv) -> None:
        '''
        Update params using gradient descent algo of all layers
        GD, SGD, mini-batch SGD, AdaGrad, RMSProp, Adam etc.
        Args: x : training data
            y: training labels
            backprop_func: function for 
        '''
    
        (self.weights, self.biases) = optimzation_func(self,self.lr_0, x,y, self.weights, self.biases,
                                                     self.batch_size, cost_func_deriv)
        

        return None


    def backprop(self, zs, activations, y, cost_func_deriv, activation_func_deriv, output_unit_deriv) -> tuple[np.ndarray, np.ndarray]:
        '''
         Calculate gradients of pre-activations (x.W+b) for each layer
         Args: zs : pre-activation outputs
            activations : post-activation outputs
            y : training labels
        '''

        w_gradient = [np.zeros_like(w_i) for w_i in self.weights] # i represents ith layer
        b_gradient = [np.zeros_like(b_i) for b_i in self.biases]        

        g = cost_func_deriv(activations[-1],y)

        g = g * output_unit_deriv(zs[-1]) #(1,10)
        w_gradient[-1] = np.dot(activations[-2].T,g) # Transpose and g.hT in OG eq   0,1,2 ; (30,1).(1,10) = (30,10)
        b_gradient[-1] = g # (1,10)
        g = np.dot(g, self.weights[-1].T) # (1,10).(10,30)=(1,30)

        for layer in range(2,len(self.model_dim)):
            g = g * activation_func_deriv(zs[-layer]) #(1,30)

            w_gradient[-layer] = np.dot(activations[-(layer+1)].T,g) #(1,784).(1,30)=(784,30)
            b_gradient[-layer] = g
            g = np.dot(g, self.weights[-layer].T)

            
        return (w_gradient,b_gradient)

    def evaluate(self, x: np.ndarray, y: np.ndarray) -> None:
        '''
        Evaluate model on validation or test set
        '''
        
        predictions = []
        for x_i in x:
            activation = x_i
            
            for layer in range(1,len(self.model_dim)): # 0 idx is input
                activation = self.forwardpass( activation, layer, sigmoid, sigmoid, mode= 'eval')
            
            predictions.append(np.argmax(activation))
            
        
        predictions = np.asarray(predictions)
        
        #calculate loss
        val_loss = MSE(predictions,y)

        print(f'Test/validation loss: {val_loss}')
        print(predictions,y) #BUG: all predictions are same. perhaps check forwardpass for eval.
         
        return sum(int(pred==y_i) for pred,y_i in zip(predictions,y))

    def fit(self, train_dataloader: tuple[np.ndarray,np.ndarray], eval_dataloader: tuple[np.ndarray,np.ndarray],
            lr_0: float, max_epochs: int = 1 , batch_size: int = 10, evaluate_every_n_epochs: int = 1) -> None:

        train_x, train_y = train_dataloader
        val_x, val_y = eval_dataloader
        self.evaluate_every_n_epochs = evaluate_every_n_epochs
        self.max_epochs = max_epochs
        self.lr_0 = lr_0
        self.batch_size = batch_size
    
        if train_x.shape[1] != self.model_dim[0] or train_y.shape[1] != self.model_dim[-1]:
            print(f"training dataset not in correct dimension. \
                  Needed {self.model_dim[0],self.model_dim[-1]} \
                    received {train_x.shape[1], train_y.shape[1]}")

        for epoch in range(self.max_epochs): 

            self.backwardpass(train_x, train_y, minibatchSGD, MSE_derivative)
            print(f'Epoch {epoch+1}/{self.max_epochs} done')
            
            if epoch % evaluate_every_n_epochs == 0: 
                print(f'Evaluation: {self.evaluate(val_x, val_y)}/{val_y.shape[0]} correct')
                
        return None



## Helper functions

In [30]:
def sigmoid(x: np.ndarray) -> np.ndarray:
    '''
    Sigmoid function to a layer of NN
    '''
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_derivative(x: np.ndarray) -> np.ndarray:
    '''
    Sigmoid function derivative
    '''
    return sigmoid(x) * (1.0 - sigmoid(x))

def ReLu(x: np.ndarray) -> np.ndarray:
    '''
    Relu function to a layer of NN
    '''
    return np.maximum(0,x)

def ReLu_derivative(x: np.ndarray) -> np.ndarray:
    '''
    Relu function derivative 
    '''
    # if x.all() > 0: #BUG: is .all correct??
    #     return 1
    # else: return 0
    return (x>0)*1

    

def MSE(y: np.ndarray, y_true: np.ndarray) -> np.ndarray:
    '''
    (NOT USED)
    Cost function - Mean Squared Error
    Args: y: predictions/ activations from output units
        y_true: data labels 
    '''
    n = y.shape[0]
    cost = np.sum(np.absolute(y-y_true)**2)/(2.0*n)
    return cost

def MSE_derivative(y: np.ndarray, y_true: np.ndarray) -> np.ndarray:
    '''
    Args: y: predictions
        y_true: data labels 
    '''
    return y - y_true

def BGD(self, epsilon_fixed, x: np.ndarray, y: np.ndarray, w,b, cost_func_deriv) -> tuple[np.ndarray,np.ndarray]:
    '''
    Batched gradient decent algorithm
    '''
    batch_size = x.shape[0]

    trainset = list(zip(x,y))
    np.random.shuffle(trainset) #shuffle in-place


    activations = [np.zeros((1,i)) for i in self.model_dim] # including input and output of the network
    zs = [np.zeros((1,i)) for i in self.model_dim] # including input and output of the network
    w_gradient = [np.zeros_like(w_i) for w_i in w] # i represents ith layer
    b_gradient = [np.zeros_like(b_i) for b_i in b]        

    # for each sample
    for x, y in trainset:
        activations[0] = x.reshape(1,-1)
        zs[0] = x.reshape(1,-1)
        
        # forward pass
        for layer in range(1,len(self.model_dim)): # 0 idx is input
                zs[layer], activations[layer] = self.forwardpass( activations[layer-1], layer, sigmoid, sigmoid)

        # backprop
        delta_w_gradient, delta_b_gradient = self.backprop(zs,activations,y, cost_func_deriv, sigmoid_derivative, sigmoid_derivative)
        
        # accumulate gradients before applying
        w_gradient = [wg_i + d_wg_i for wg_i, d_wg_i in zip(w_gradient,delta_w_gradient)]
        b_gradient = [bg_i + d_bg_i for bg_i, d_bg_i in zip(b_gradient,delta_b_gradient)]

    # apply gradients
    w = [w_i - (epsilon_fixed/batch_size) * w_gradient_i for w_i, w_gradient_i in zip(w, w_gradient)]
    b = [b_i - (epsilon_fixed/batch_size) * b_gradient_i for b_i, b_gradient_i in zip(b, b_gradient)]
    
    # TODO: evaluate_every_n_steps here

    return (w, b)

setattr(Network, 'BGD', BGD)


def minibatchSGD(self, epsilon_fixed, x: np.ndarray, y: np.ndarray, 
                 w, b, batch_size, cost_func_deriv ) -> tuple[list[np.ndarray], list[np.ndarray]]:


    trainset = list(zip(x,y))
    np.random.shuffle(trainset) #shuffle in-place
    
    mini_batches = [ trainset[i:i+batch_size] for i in range(0, len(trainset), batch_size)]
    
    for mini_batch in mini_batches:

        mini_batch_x, mini_batch_y = zip(*mini_batch)
        mini_batch_x = np.stack(mini_batch_x, axis = 0)
        mini_batch_y = np.stack(mini_batch_y, axis = 0)
        
        (w, b) = BGD(self, epsilon_fixed, mini_batch_x, mini_batch_y, w, b, cost_func_deriv)
    return (w, b)
# setattr(Network, 'minibatchSGD', minibatchSGD)



    

## Data Loader

In [29]:
def load_mnist() -> tuple[tuple,tuple,tuple]:
    '''
    Args: None
    Return: tuple(train_x,train_y), tuple(val_x,val_y), tuple(test_x,test_y) 

    train data -> 60k
    test data -> 10k
    image -> 28x28 = 728
    train_x -> (50k,728) , train_y -> (50k,10)
    val_x -> (10k,728), val_y -> (10k,10)
    test_x -> (10k,728), test_y -> (10k,1)

    '''
    (trainX, trainy), (testX, testy) = load_data() #TODO: shuffle trainX

    # train_data = [np.reshape(img,(784,1)) for img in trainX]  # 60k x (784,1), if imaginging NN arch horizontal
    trainX = np.reshape(trainX,(-1,784))/255.0 # 60k x (1,784) = 60k x 784, if imagining NN arch vertical. ie row vector matrices
    testX = np.reshape(testX,(-1,784))/255.0 # 10k x (1,784) = 10k x 784

    train_data, train_label = trainX[:50000], trainy[:50000]
    val_data, val_label = trainX[50000:], trainy[50000:]

    train_label = np.array([vectorize_digit(label) for label in train_label])
    # train_label = train_label.reshape(50000,10)
    # val_label = val_label.reshape(10000,1)
    # testy = testy.reshape(10000,1)
    
    print(train_data.shape,train_label.shape)
    print(val_data.shape,val_label.shape)
    print(testX.shape,testy.shape)
    return ((train_data,train_label),(val_data, val_label), (testX,testy) )


def vectorize_digit(digit: np.uint) -> np.ndarray:
    '''
    Args: digit like 0-9
    Return: vectorized form. 7 becomes [0,0,0,0,0,0,0,1,0,0]
    '''
    d = np.zeros(10)
    d[digit] = 1.0
    
    return d

## Main

In [42]:
if __name__ == '__main__':

    (trainset,valset,testset) = load_mnist()
    model = Network(model_dim=[trainset[0].shape[-1],30,trainset[1].shape[-1]])
    model.fit(trainset, testset, lr_0 = 0.002,  max_epochs= 30, batch_size=16) #note validating on test set
    
  #TODO: try trainset as validation set.
    

(50000, 784) (50000, 10)
(10000, 784) (10000,)
(10000, 784) (10000,)
Epoch 1/30 done
Test/validation loss: 5.7433
[6 6 6 ... 4 4 6] [7 2 1 ... 4 5 6]
Evaluation: 998/10000 correct
Epoch 2/30 done
Test/validation loss: 8.5998
[7 6 7 ... 0 8 9] [7 2 1 ... 4 5 6]
Evaluation: 1149/10000 correct
Epoch 3/30 done
Test/validation loss: 8.2284
[7 6 7 ... 0 8 9] [7 2 1 ... 4 5 6]
Evaluation: 1250/10000 correct
Epoch 4/30 done
Test/validation loss: 8.05395
[7 6 7 ... 0 8 9] [7 2 1 ... 4 5 6]
Evaluation: 1321/10000 correct
Epoch 5/30 done
Test/validation loss: 7.9247
[7 6 7 ... 0 8 9] [7 2 1 ... 4 5 6]
Evaluation: 1372/10000 correct
Epoch 6/30 done
Test/validation loss: 7.86455
[7 6 7 ... 0 8 9] [7 2 1 ... 4 5 6]
Evaluation: 1400/10000 correct
Epoch 7/30 done
Test/validation loss: 7.78925
[7 6 7 ... 5 8 9] [7 2 1 ... 4 5 6]
Evaluation: 1426/10000 correct
Epoch 8/30 done
Test/validation loss: 7.7192
[7 6 7 ... 5 8 9] [7 2 1 ... 4 5 6]
Evaluation: 1459/10000 correct
Epoch 9/30 done
Test/validation l

TODO:
* Try making all vectors as column vectors, like done in book.
* Print loss at each epoch for train set.
* (DONE) Print loss at each epoch for val set.
* Check weight/bias arrays are updating after each step.

Observations:
* NN doesnt necessarily has same predictions for consequtive epochs. 
    * activation of output unit is all 0s except one(which is also -6 to -100 decimal place). why are figures absolutely 0? for Sigmoid func, this means most of the pred are around the centre.
* delta_w_gradient from backprop for 784x30 nodes are 0.00, hence first layer is not learning, but 2nd is.
    * possibly the loop on layers in incorrect. YES.
    * shapes are incorrect during backprop. YES.
    * Not all are 0. it is learning, but has most of them 0.00.
* loss and correct predictions have no correlation. sometimes, higher loss for more correct predictions sometimes low.

* Hyperparameter tuning:
    * lr_0 = 0.005, batch_size = 10, epochs=5: shows different predictions and improves.
    * lr_0 = 0.0005 with batch_size = 10, epochs=5: shows same predictions but improves faster.
    * lr_0 = 0.0005 with batch_size = 10, epochs=20: 2645/10k.
    * lr_0 = 0.0005 with batch_size = 10, epochs=50: 1673/10k.
    * lr_0 = 0.005 with batch_size = 10, epochs=20: 1416/10k.
    * lr_0 = 0.005 with batch_size = 10, epochs=50: 2173/10k.
    * lr_0 = 0.001 with batch_size = 10, epochs=30: 2665/10k.
    * lr_0 = 0.0015 with batch_size = 10, epochs=30: 2756/10k.
    * lr_0 = 0.002 with batch_size = 10, epochs=30: 4168/10k.
    * lr_0 = 0.0025 with batch_size = 10, epochs=30: 2770/10k.
    * lr_0 = 0.003 with batch_size = 10, epochs=30: 1739/10k.
    * lr_0 = 0.002 with batch_size = 10, epochs=100: 6755/10k.


## playground

In [59]:
%%timeit
# b_1 = np.random.randn(1,30)
# w_1 = np.random.randn(784,30)
# x_1 = np.random.randn(1,784)
# z_1 = np.dot(x_1,w_1)+b_1
# derv_1 = sigmoid_derivative(z_1)
# print(z_1.shape, z_1)


b_2 = np.random.randn(30,1)
w_2 = np.random.randn(30,784)
x_2 = np.random.randn(784,1)
z_2 = np.dot(w_2, x_2)+b_2
derv_2 = sigmoid_derivative(z_2)
# print(z_2.shape, z_2)
# print(derv_2)

899 µs ± 36.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
